# Data Parallelism


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [4]:
# parameters
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

In [7]:
print(device)

cuda:0


## Dummy Dataset
Making a dummy (random) dataset.

In [20]:
class RandomDataset(Dataset):
    '''
    Creates a radomly generated Dataset.    
    ---args---
    size(int): specifies the size of the dataset.
    length(int): specifies the length of the dataset.
    '''
    
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

In [21]:
rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                        batch_size=batch_size, shuffle=True)

In [22]:
print(type(rand_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


## Simple Model
Our model just gets an input, performs a linear operation, and gives an output. However, you can use DataParallel on any model (CNN, RNN, Capsule Net etc.)

In [29]:
class Model(nn.Module):
    
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
    
    def forward(self, input):
        output = self.fc(input)
        print('\tIn Model: input size', input.size(),
             "output size", output.size())
        
        return output

## Create Model and Data Parallel
First, we need to make a model instance and check if we have multiple GPUs. If we have multiple GPUs, we can wrap our model using ```nn.DataParallel```. Then we can put our model on GPUs by ```model.to(device)```.

In [30]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print(torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
else:
    print("There's just one GPU!")

model.to(device)

There's just one GPU!


Model(
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

## Run the Model

In [31]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Output: input size", input.size(),
         "output_size", output.size())

	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Output: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Output: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([30, 5]) output size torch.Size([30, 2])
Output: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Output: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
